<a href="https://colab.research.google.com/github/Xeena2812/onallo-labor/blob/fit-to-cifar-inceptionv4/onlab/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the CIFAR-100 dataset

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.preprocessing import LabelBinarizer
# To ignore random tensorflow warnings.
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='fine')

print(x_train.shape, y_train.shape)

169001437/169001437 [==============================] - 4s 0us/step
(50000, 32, 32, 3) (50000, 1)


In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
x_train = (x_train / 255 - 0.5) * 2
x_test = (x_test / 255 - 0.5) * 2

enc = LabelBinarizer()

y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)


In [5]:
print(x_test[0].shape)
print(y_test[0].shape)

(32, 32, 3)
(100,)


## Inception v4 Implementation

The CIFAR100 dataset is composed of 32x32 pixel images as oppopsed to the 299x299 pixel images of the Imagenet network for which the Inceptionv4 network was originally comstructed. There a significant downsizing of the network is necessary to accomodate the 10x size difference.

Inception blocks are used to create feature maps, and reduction blocks are used to downsize the filter size for teh inception blocks
Intuitions:
1. Put 5x5 convolutions in stem to downsize image to smaller size, and expand filter space for the Inception-A blocks.
2. First inception blocks TRANSFORM the feature map (with 1x1 convolutions) while KEEPING the same image and filter size.
3. Reduction-A blocks further reduce the image size and expand filter space for Inception-B blocks.
4. Inception-B blocks further transform the feature map.
5. Probably not neccessary, but if it is add Reduction-B blocks
6. A final AveragePooling layer reduces  

1. Drop the reduction blocks as the image is already small adn there is no need to reduce image size (probably only dimension reduction is needed)
   1. this probably won't work as different size feature extraction is necessary
2. Reduce the number of Conv2DBatchNorm layers in each block (Inception, Reduction, Stem) in order not to decrease the image size down to one so quickly
3. Change all filter numbers to half.

10. Take design principles outlaid in the v3 paper into account

In [6]:
#import tensorflow as tf
from keras.layers import Dense, Conv2D, Input, MaxPooling2D, BatchNormalization, Activation, AveragePooling2D, Dropout, Concatenate, Flatten
#from keras.layers import *
from keras import Model
from keras.optimizers import RMSprop, Adam


# TODO: Go to Inceptionv2 paper to understand Conv2D_BN and fine tune parameters potentially
def Conv2DBatchNorm(x, filters, kernel_size=(3, 3), strides=(1, 1), padding='same'):

    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

def Stem(x):
    print("pre-stem size:", x.shape)

    x = Conv2DBatchNorm(x, 16, (5, 5))
    x = Conv2DBatchNorm(x, 32, (3, 3))

    branch_0 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)
    branch_1 = Conv2DBatchNorm(x, 48, (3, 3), strides=(2, 2), padding='valid')

    x = Concatenate()([branch_0, branch_1])
    
    branch_0 = Conv2DBatchNorm(x, 32, (1, 1))
    branch_0 = Conv2DBatchNorm(branch_0, 48, (3, 3), padding='valid')

    branch_1 = Conv2DBatchNorm(x, 32, (1, 1))
    branch_1 = Conv2DBatchNorm(branch_1, 32, (1, 7))
    branch_1 = Conv2DBatchNorm(branch_1, 32, (7, 1))
    branch_1 = Conv2DBatchNorm(branch_1, 48, (3, 3), padding='valid')

    x = Concatenate()([branch_0, branch_1])
    
    branch_0 = Conv2DBatchNorm(x, 96, (3, 3))

    # Here another reduction by half should occur according too inceptionv4, but it's too big a reduction so it's left off.
    branch_1 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)

    x = Concatenate()([branch_0, branch_1])

    print("post-stem size:", x.shape)

    return x


def InceptionA(x):
    print("pre-A size:", x.shape)

    branch_0 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_0 = Conv2DBatchNorm(branch_0, 48, (1, 1))

    branch_1 = Conv2DBatchNorm(x, 48, (1, 1))

    branch_2 = Conv2DBatchNorm(x, 32, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 48, (3, 3))

    branch_3 = Conv2DBatchNorm(x, 32, (1, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 48, (3, 3))
    branch_3 = Conv2DBatchNorm(branch_3, 48, (3, 3))

    x = Concatenate()([branch_0, branch_1, branch_2, branch_3])
    print("post-A size:", x.shape)

    return x


def InceptionB(x):
    print("pre-B size:", x.shape)

    branch_0 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_0 = Conv2DBatchNorm(branch_0, 64, (1, 1))

    branch_1 = Conv2DBatchNorm(x, 192, (1, 1))

    branch_2 = Conv2DBatchNorm(x, 96, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 112, (1, 7))
    branch_2 = Conv2DBatchNorm(branch_2, 128, (7, 1))

    branch_3 = Conv2DBatchNorm(x, 96, (1, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 96, (1, 7))
    branch_3 = Conv2DBatchNorm(branch_3, 112, (7, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 112, (1, 7))
    branch_3 = Conv2DBatchNorm(branch_3, 128, (7, 1))

    x = Concatenate()([branch_0, branch_1, branch_2, branch_3])
    print("post-B size:", x.shape)

    return x


def InceptionC(x):
    print("pre-C size:", x.shape)

    branch_0 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_0 = Conv2DBatchNorm(branch_0, 128, (1, 1))

    branch_1 = Conv2DBatchNorm(x, 128, (1, 1))

    branch_2 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_2_0 = Conv2DBatchNorm(branch_2, 128, (1, 3))
    branch_2_1 = Conv2DBatchNorm(branch_2, 128, (3, 1))

    branch_3 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 224, (1, 3))
    branch_3 = Conv2DBatchNorm(branch_3, 256, (3, 1))
    branch_3_0 = Conv2DBatchNorm(branch_3, 128, (3, 1))
    branch_3_1 = Conv2DBatchNorm(branch_3, 128, (1, 3))

    x = Concatenate()([branch_0, branch_1, branch_2_0, branch_2_1, branch_3_0, branch_3_1])
    print("post-C size:", x.shape)

    return x


def ReductionA(x):
    print("pre-RedA size:", x.shape)

    branch_0 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    branch_1 = Conv2DBatchNorm(x, 192, (3, 3), strides=(2, 2), padding='valid')

    branch_2 = Conv2DBatchNorm(x, 96, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 112, (3, 3))
    branch_2 = Conv2DBatchNorm(branch_2, 128, (3, 3), strides=(2, 2), padding='valid')

    x = Concatenate()([branch_0, branch_1, branch_2])
    print("post-RedA size:", x.shape)

    return x


def ReductionB(x):
    print("pre-RedB size:", x.shape)

    branch_0 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

    branch_1 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_1 = Conv2DBatchNorm(branch_1, 192, (2, 2), strides=(2, 2), padding='valid')

    branch_2 = Conv2DBatchNorm(x, 256, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 256, (1, 7))
    branch_2 = Conv2DBatchNorm(branch_2, 320, (7, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 320, (2, 2), strides=(2, 2), padding='valid')

    x = Concatenate()([branch_0, branch_1, branch_2])
    print("post-RedB size:", x.shape)

    return x


def MyInception():
    inputs = Input(shape=(32, 32, 3), name='input')

    x = Stem(inputs)

    for _ in range(2):
        x = InceptionA(x)

    x = ReductionA(x)

    for _ in range(3):
        x = InceptionB(x)

    x = ReductionB(x)

    #for _ in range(3):
    x = InceptionC(x)

    x = AveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    outputs = Dense(100, name='output')(x)
    model = Model(inputs=inputs, outputs=outputs, name='MyInception')

    return model


model = MyInception()




pre-stem size: (None, 32, 32, 3)
post-stem size: (None, 13, 13, 192)
pre-A size: (None, 13, 13, 192)
post-A size: (None, 13, 13, 192)
pre-A size: (None, 13, 13, 192)
post-A size: (None, 13, 13, 192)
pre-RedA size: (None, 13, 13, 192)
post-RedA size: (None, 6, 6, 512)
pre-B size: (None, 6, 6, 512)
post-B size: (None, 6, 6, 512)
pre-B size: (None, 6, 6, 512)
post-B size: (None, 6, 6, 512)
pre-B size: (None, 6, 6, 512)
post-B size: (None, 6, 6, 512)
pre-RedB size: (None, 6, 6, 512)
post-RedB size: (None, 3, 3, 1024)
pre-C size: (None, 3, 3, 1024)
post-C size: (None, 3, 3, 768)


In [9]:
from keras.metrics import TopKCategoricalAccuracy
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=[
                  'categorical_accuracy',
                  TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
                  TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
              ])

print("Number of paramteres:", model.count_params())

Number of paramteres: 6319316


Notes:

The Stem Reduces image size from 299 to 35 in the original Inceptionv4 architecture which is a 1/10 scale. That is too large of a downscale and we can't afford that with a 32 by 32 image. That's why my stem only reduces image 1/3. I've also reduced all teh filter numbers to half their original number, because presumably for a smaller mage a smaller repsresentation will be sufficient

Questions:
What metrics to use
What to log
How many blocks to use

In [10]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

model.fit(x_train, y_train,
          batch_size=32,
          epochs=200,
          validation_split=0.2,
          validation_batch_size=32,
          callbacks=[
              TensorBoard(
                  log_dir='./logs',
              ),
              EarlyStopping(
                  monitor='val_categorical_accuracy',
                  patience=10,
              ),
              ModelCheckpoint(
                  './checkpoint',
                  monitor='val_categorical_accuracy',

              )
          ],
          )


Epoch 1/200
1249/1250 [============================>.] - ETA: 0s - loss: 8.6386 - categorical_accuracy: 0.0159 - top_1_accuracy: 0.0159 - top_5_accuracy: 0.0730

1250/1250 [==============================] - 125s 77ms/step - loss: 8.6385 - categorical_accuracy: 0.0159 - top_1_accuracy: 0.0159 - top_5_accuracy: 0.0730 - val_loss: 8.5684 - val_categorical_accuracy: 0.0220 - val_top_1_accuracy: 0.0220 - val_top_5_accuracy: 0.1008
Epoch 2/200
1250/1250 [==============================] - ETA: 0s - loss: 8.6465 - categorical_accuracy: 0.0190 - top_1_accuracy: 0.0190 - top_5_accuracy: 0.0893

1250/1250 [==============================] - 95s 76ms/step - loss: 8.6465 - categorical_accuracy: 0.0190 - top_1_accuracy: 0.0190 - top_5_accuracy: 0.0893 - val_loss: 8.5635 - val_categorical_accuracy: 0.0224 - val_top_1_accuracy: 0.0224 - val_top_5_accuracy: 0.1003
Epoch 3/200
1250/1250 [==============================] - ETA: 0s - loss: 8.0349 - categorical_accuracy: 0.0199 - top_1_accuracy: 0.0199 - top_5_accuracy: 0.0914

1250/1250 [==============================] - 93s 74ms/step - loss: 8.0349 - categorical_accuracy: 0.0199 - top_1_accuracy: 0.0199 - top_5_accuracy: 0.0914 - val_loss: 7.4869 - val_categorical_accuracy: 0.0207 - val_top_1_accuracy: 0.0207 - val_top_5_accuracy: 0.1088
Epoch 4/200
1249/1250 [============================>.] - ETA: 0s - loss: 7.9351 - categorical_accuracy: 0.0219 - top_1_accuracy: 0.0219 - top_5_accuracy: 0.0940

1250/1250 [==============================] - 90s 72ms/step - loss: 7.9372 - categorical_accuracy: 0.0219 - top_1_accuracy: 0.0219 - top_5_accuracy: 0.0940 - val_loss: 8.2895 - val_categorical_accuracy: 0.0216 - val_top_1_accuracy: 0.0216 - val_top_5_accuracy: 0.1020
Epoch 5/200
1249/1250 [============================>.] - ETA: 0s - loss: 8.3016 - categorical_accuracy: 0.0232 - top_1_accuracy: 0.0232 - top_5_accuracy: 0.0995

...

1250/1250 [==============================] - 92s 74ms/step - loss: 8.0578 - categorical_accuracy: 0.0249 - top_1_accuracy: 0.0249 - top_5_accuracy: 0.1145 - val_loss: 8.0784 - val_categorical_accuracy: 0.0266 - val_top_1_accuracy: 0.0266 - val_top_5_accuracy: 0.1185
Epoch 43/200
1249/1250 [============================>.] - ETA: 0s - loss: 7.9429 - categorical_accuracy: 0.0256 - top_1_accuracy: 0.0256 - top_5_accuracy: 0.1137

1250/1250 [==============================] - 93s 74ms/step - loss: 7.9426 - categorical_accuracy: 0.0256 - top_1_accuracy: 0.0256 - top_5_accuracy: 0.1137 - val_loss: 8.0961 - val_categorical_accuracy: 0.0262 - val_top_1_accuracy: 0.0262 - val_top_5_accuracy: 0.1179
Epoch 44/200
1249/1250 [============================>.] - ETA: 0s - loss: 8.0421 - categorical_accuracy: 0.0239 - top_1_accuracy: 0.0239 - top_5_accuracy: 0.1152

1250/1250 [==============================] - 92s 74ms/step - loss: 8.0437 - categorical_accuracy: 0.0239 - top_1_accuracy: 0.0239 - top_5_accuracy: 0.1152 - val_loss: 8.0800 - val_categorical_accuracy: 0.0265 - val_top_1_accuracy: 0.0265 - val_top_5_accuracy: 0.1182
Epoch 45/200
1249/1250 [============================>.] - ETA: 0s - loss: 8.0377 - categorical_accuracy: 0.0256 - top_1_accuracy: 0.0256 - top_5_accuracy: 0.1135

1250/1250 [==============================] - 91s 73ms/step - loss: 8.0361 - categorical_accuracy: 0.0256 - top_1_accuracy: 0.0256 - top_5_accuracy: 0.1135 - val_loss: 8.0929 - val_categorical_accuracy: 0.0268 - val_top_1_accuracy: 0.0268 - val_top_5_accuracy: 0.1180
Epoch 46/200
1038/1250 [=======================>......] - ETA: 9s - loss: 8.0382 - categorical_accuracy: 0.0251 - top_1_accuracy: 0.0251 - top_5_accuracy: 0.1143 

KeyboardInterrupt: ignored